# **Pré-processamento**

In [16]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import pandas as pd
import seaborn as sns
import missingno as msno
import warnings
from matplotlib import pyplot as plt
import datetime
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)

In [18]:
df = pd.read_csv("gdrive/My Drive/dissertação/Qualidade_do_ar_-_Dados_horarios.csv",sep=",")

In [19]:
 df['Data']=df['Data'].astype('str')

In [20]:
df['Data']=df['Data'].apply(lambda x: x[:-3])

In [21]:
#conversão para datetime
def convert_to_datetime(x):
    return(datetime.datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))
df['Data']=df['Data'].apply(convert_to_datetime) 
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-8:-3])
# #extrair ano, mes, hora 
df['Ano']=df['Data'].apply(lambda x: x.year)
df['Mês']=df['Data'].apply(lambda x: x.month)
df['Dia']=df['Data'].apply(lambda x: x.day)
df['Hora']=df['Data'].apply(lambda x: x.hour)
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-11:-6])

In [22]:
df_bg=df[df['Estação']=='BG'] #Bangu
df_gc=df[df['Estação']=='CG'] #Campo Grande
df_ca=df[df['Estação']=='CA'] #Centro
df_av=df[df['Estação']=='AV'] #Copacabana
df_ir=df[df['Estação']=='IR'] #Irajá
df_pg=df[df['Estação']=='PG'] #Pedra de Guaratiba
df_sc=df[df['Estação']=='SC'] #São Cristóvão
df_sp=df[df['Estação']=='SP'] #Tijuca

In [23]:
df_sc.set_index('Data',inplace=True)

In [24]:
df_sc.drop(columns=['OBJECTID','CodNum','Estação','Dir_Vento','NO2','HCNM','HCT','CH4','NO','NOx',\
                   'PM2_5', 'Lat', 'Lon', 'X_UTM_Sirgas2000','Y_UTM_Sirgas2000'], inplace=True)

In [25]:
df_sc_2=df_sc.copy() #para fazer distribuição sem eliminar ausentes

In [26]:
df_sc=df_sc['2011-01-01':'2018-01-01']

In [27]:
#explicar que usou interpolação linear com spline e justificar
df_sc['Chuva'].interpolate(method='slinear', inplace=True)
df_sc['Pres'].interpolate(method='slinear', inplace=True)
df_sc['RS'].interpolate(method='slinear', inplace=True)
df_sc['Temp'].interpolate(method='slinear', inplace=True)
df_sc['UR'].interpolate(method='slinear', inplace=True)
df_sc['Vel_Vento'].interpolate(method='slinear', inplace=True)
df_sc['SO2'].interpolate(method='slinear', inplace=True)
df_sc['CO'].interpolate(method='slinear', inplace=True)
df_sc['O3'].interpolate(method='slinear', inplace=True)
df_sc['PM10'].interpolate(method='slinear', inplace=True)

In [28]:
df_sc

,Chuva,Pres,RS,Temp,UR,Vel_Vento,SO2,CO,O3,PM10,Hora-minuto,Ano,Mês,Dia,Hora
Data,,,,,,,,,,,,,,,
2011-01-01 00:30:00,0.0,1009.60,2.87,26.66,90.73,0.47,4.09,0.03,8.47,8.0,01 00,2011,1,1,0
2011-01-01 01:30:00,0.0,1008.78,3.02,26.26,93.75,0.52,5.08,0.04,4.43,17.0,01 01,2011,1,1,1
2011-01-01 02:30:00,0.2,1008.57,2.72,25.80,97.58,0.48,4.85,0.04,6.19,38.0,01 02,2011,1,1,2
2011-01-01 03:30:00,0.0,1008.33,1.98,25.04,99.83,0.60,1.54,0.02,9.96,17.0,01 03,2011,1,1,3
2011-01-01 04:30:00,0.2,1008.50,1.63,25.05,99.83,0.43,2.16,0.04,2.17,10.0,01 04,2011,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-01 19:30:00,0.0,1008.27,1.00,28.23,70.30,1.15,5.72,0.04,54.62,12.0,01 19,2018,1,1,19
2018-01-01 20:30:00,0.0,1008.85,1.00,28.25,68.78,1.05,5.15,0.07,43.37,11.0,01 20,2018,1,1,20
2018-01-01 21:30:00,0.0,1009.48,1.00,28.15,67.30,0.52,4.96,0.07,34.61,10.0,01 21,2018,1,1,21


#**Gridsearch**

In [29]:
df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)

In [30]:
df_sc.head(1)

,Chuva,Pres,RS,Temp,UR,Vel_Vento,SO2,CO,O3,PM10
Data,,,,,,,,,,
2011-01-01 00:30:00,0.0,1009.6,2.87,26.66,90.73,0.47,4.09,0.03,8.47,8.0


In [31]:
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc, df_sc_copy], axis=1, ignore_index=False)

In [32]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [33]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [34]:
#gerando valores a partir do dataframe comum
df_sc.head(1)

,Chuva,Pres,RS,Temp,UR,Vel_Vento,SO2,CO,PM10,O3
Data,,,,,,,,,,
2011-01-01 00:30:00,0.0,1009.6,2.87,26.66,90.73,0.47,4.09,0.03,8.0,8.47


In [35]:
valores=df_sc.values

In [36]:
valores = valores.astype('float32')

In [37]:
df_sc.head(1)

,Chuva,Pres,RS,Temp,UR,Vel_Vento,SO2,CO,PM10,O3
Data,,,,,,,,,,
2011-01-01 00:30:00,0.0,1009.6,2.87,26.66,90.73,0.47,4.09,0.03,8.0,8.47


In [38]:
# normalize features
scaler= MinMaxScaler(feature_range=(0, 1))
scaled= scaler.fit_transform(valores)

In [39]:
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)

In [40]:
reframed

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t),var7(t),var8(t),var9(t),var10(t)
1,0.000000,0.315287,0.002743,0.453304,0.894683,0.094382,0.031866,0.006342,0.032653,0.034645,0.000000,0.290899,0.002886,0.443550,0.928993,0.105618,0.039579,0.008457,0.069388,0.018120
2,0.000000,0.290899,0.002886,0.443550,0.928993,0.105618,0.039579,0.008457,0.069388,0.018120,0.003584,0.284653,0.002600,0.432334,0.972506,0.096629,0.037787,0.008457,0.155102,0.025319
3,0.003584,0.284653,0.002600,0.432334,0.972506,0.096629,0.037787,0.008457,0.155102,0.025319,0.000000,0.277514,0.001892,0.413801,0.998069,0.123596,0.011998,0.004228,0.069388,0.040740
4,0.000000,0.277514,0.001892,0.413801,0.998069,0.123596,0.011998,0.004228,0.069388,0.040740,0.003584,0.282570,0.001558,0.414045,0.998069,0.085393,0.016829,0.008457,0.040816,0.008876
5,0.003584,0.282570,0.001558,0.414045,0.998069,0.085393,0.016829,0.008457,0.040816,0.008876,0.000000,0.288519,0.007006,0.418191,0.997955,0.085393,0.020647,0.014799,0.077551,0.009633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60356,0.000000,0.251339,0.013543,0.519629,0.534765,0.400000,0.037476,0.004228,0.024490,0.227831,0.000000,0.275728,0.000956,0.491587,0.662577,0.247191,0.044566,0.008457,0.048980,0.223413
60357,0.000000,0.275728,0.000956,0.491587,0.662577,0.247191,0.044566,0.008457,0.048980,0.223413,0.000000,0.292980,0.000956,0.492075,0.645308,0.224719,0.040125,0.014799,0.044898,0.177397
60358,0.000000,0.292980,0.000956,0.492075,0.645308,0.224719,0.040125,0.014799,0.044898,0.177397,0.000000,0.311718,0.000956,0.489637,0.628494,0.105618,0.038644,0.014799,0.040816,0.141566
60359,0.000000,0.311718,0.000956,0.489637,0.628494,0.105618,0.038644,0.014799,0.040816,0.141566,0.000000,0.317667,0.000956,0.492807,0.597591,0.044944,0.034982,0.021142,0.057143,0.107044


In [41]:
reframed.drop(reframed.columns[[10,11,12,13,14,15,16,17,18]], axis=1, inplace=True)
print(reframed.head())

   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)  var6(t-1)  \
1   0.000000   0.315287   0.002743   0.453304   0.894683   0.094382   
2   0.000000   0.290899   0.002886   0.443550   0.928993   0.105618   
3   0.003584   0.284653   0.002600   0.432334   0.972506   0.096629   
4   0.000000   0.277514   0.001892   0.413801   0.998069   0.123596   
5   0.003584   0.282570   0.001558   0.414045   0.998069   0.085393   

   var7(t-1)  var8(t-1)  var9(t-1)  var10(t-1)  var10(t)  
1   0.031866   0.006342   0.032653    0.034645  0.018120  
2   0.039579   0.008457   0.069388    0.018120  0.025319  
3   0.037787   0.008457   0.155102    0.025319  0.040740  
4   0.011998   0.004228   0.069388    0.040740  0.008876  
5   0.016829   0.008457   0.040816    0.008876  0.009633  


In [42]:
# split into train and test sets
valores = reframed.values

In [43]:
n_train_hours = 2190 * 24 # 5 anos

In [44]:
train = valores[:n_train_hours, :] #5 anos para treino: primeiros cinco anos, todas as colunas

In [45]:
test = valores[n_train_hours:, :] #2 anos para teste: ultimos dois anos, todas as colunas

In [46]:
# split into input and outputs
train_X= train[:, :-1] #todos os registros, menos a última coluna
train_y= train[:, -1] #todos registros, somente a última coluna

#eu uso as 10 variáveis anteriores, inclusive a propria

In [47]:
test_X=test[:, :-1]
test_y=test[:, -1] 

In [48]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(52560, 1, 10) (52560,) (7800, 1, 10) (7800,)


#**teste sem gridsearch**

In [14]:
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [15]:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2]))) #50 neurônios na primeira camada
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam') 

NameError: ignored

In [ ]:
history = model.fit(train_X, train_y, epochs=5, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X) #previsões feitas com base no modelo e no teste
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2])) #(16560,10)

In [ ]:
yhat #previsões

In [ ]:
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))

In [ ]:
test_y

In [ ]:
mean_squared_error(test_y, yhat)

#**gridsearch continuação**


In [49]:
from keras.optimizers import Adam

In [50]:
# import numpy
# from sklearn.model_selection import GridSearchCV
# from keras.models import Sequential
# from keras.layers import LSTM
# from sklearn.metrics import mean_squared_error
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.wrappers.scikit_learn import KerasRegressor
# from keras.constraints import maxnorm
# from keras import metrics
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error #para obter rmse usar formula
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam
from keras.constraints import maxnorm
from keras import metrics
from sklearn.metrics import make_scorer

In [51]:
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.0005,momentum=0.4,neurons=1):
    # create model
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    optimizer = Adam(lr=learn_rate)
    # Compile model
    model.compile(loss='mae', optimizer=optimizer, metrics=[metrics.mean_squared_error,metrics.mean_absolute_error,metrics.mean_absolute_percentage_error])
    return model

In [52]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [53]:
model = KerasRegressor(build_fn=create_model, verbose=0)

In [54]:
scoring = {'mse': make_scorer(mean_squared_error),'mae':make_scorer(mean_absolute_error),'r2':make_scorer(r2_score)}

In [55]:
learn_rate=[0.0005]
neurons = [100,200,300]
batch_size = [500]
epochs = [150,200]

In [56]:
param_grid = dict(neurons=neurons, epochs=epochs,learn_rate=learn_rate,batch_size=batch_size)

In [57]:
grid = GridSearchCV(estimator=model, param_grid=param_grid,scoring=scoring,refit='mse',return_train_score=True,n_jobs=-1, cv=3)

In [58]:
grid_result = grid.fit(train_X, train_y)

In [59]:
# summarize results

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #usar se refit for uma das métricas


# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.001536 using {'batch_size': 500, 'epochs': 200, 'learn_rate': 0.0005, 'neurons': 200}


In [60]:
grid_results=pd.DataFrame(grid_result.cv_results_)
#valores são baixos pois são normalizados

In [61]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [62]:
grid_results.to_csv('data.csv')
!cp data.csv "drive/My Drive/"

In [64]:
grid_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_epochs,param_learn_rate,param_neurons,params,split0_test_mse,split1_test_mse,split2_test_mse,mean_test_mse,std_test_mse,rank_test_mse,split0_train_mse,split1_train_mse,split2_train_mse,mean_train_mse,std_train_mse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,split0_train_mae,split1_train_mae,split2_train_mae,mean_train_mae,std_train_mae,split0_test_r2,split1_test_r2,split2_test_r2,mean_test_r2,std_test_r2,rank_test_r2,split0_train_r2,split1_train_r2,split2_train_r2,mean_train_r2,std_train_r2
0,57.547768,4.313872,0.602834,0.048697,500,150,0.0005,100,"{'batch_size': 500, 'epochs': 150, 'learn_rate...",0.001627,0.001161,0.001756,0.001514,0.000256,3,0.001431,0.001614,0.001354,0.001466,0.000109,0.024732,0.021036,0.025565,0.023777,0.001968,3,0.023290,0.024492,0.022162,0.023315,0.000951,0.855394,0.859000,0.847974,0.854123,0.004590,4,0.856339,0.858579,0.861309,0.858742,0.002032
1,52.341384,0.198983,0.566777,0.061462,500,150,0.0005,200,"{'batch_size': 500, 'epochs': 150, 'learn_rate...",0.001625,0.001159,0.001748,0.001511,0.000254,5,0.001435,0.001627,0.001350,0.001471,0.000116,0.024334,0.020888,0.025554,0.023592,0.001976,5,0.022950,0.024520,0.022215,0.023228,0.000961,0.855528,0.859204,0.848638,0.854457,0.004380,1,0.855951,0.857460,0.861670,0.858361,0.002420
2,52.895206,0.600300,0.586503,0.056254,500,150,0.0005,300,"{'batch_size': 500, 'epochs': 150, 'learn_rate...",0.001629,0.001178,0.001718,0.001508,0.000237,6,0.001434,0.001609,0.001345,0.001463,0.000110,0.024891,0.021234,0.025530,0.023885,0.001893,2,0.023413,0.024538,0.022513,0.023488,0.000828,0.855191,0.856924,0.851238,0.854451,0.002379,2,0.856023,0.859055,0.862196,0.859091,0.002520
3,69.192923,0.499455,0.646441,0.066132,500,200,0.0005,100,"{'batch_size': 500, 'epochs': 200, 'learn_rate...",0.001627,0.001158,0.001748,0.001511,0.000254,4,0.001436,0.001618,0.001348,0.001467,0.000113,0.024218,0.020930,0.025599,0.023582,0.001958,6,0.022850,0.024499,0.022109,0.023153,0.000999,0.855322,0.859265,0.848693,0.854427,0.004362,3,0.855909,0.858264,0.861894,0.858689,0.002462
4,69.288942,0.323416,0.590065,0.038532,500,200,0.0005,200,"{'batch_size': 500, 'epochs': 200, 'learn_rate...",0.001610,0.001170,0.001829,0.001536,0.000274,1,0.001424,0.001649,0.001390,0.001488,0.000115,0.024293,0.021174,0.026518,0.023995,0.002192,1,0.022980,0.024894,0.022746,0.023540,0.000962,0.856841,0.857842,0.841665,0.852116,0.007401,6,0.857063,0.855578,0.857549,0.856730,0.000838
5,66.268884,3.871607,0.547312,0.154152,500,200,0.0005,300,"{'batch_size': 500, 'epochs': 200, 'learn_rate...",0.001628,0.001163,0.001760,0.001517,0.000256,2,0.001440,0.001608,0.001357,0.001468,0.000105,0.024207,0.021059,0.025706,0.023657,0.001937,4,0.022913,0.024463,0.022445,0.023274,0.000862,0.855262,0.858742,0.847650,0.853884,0.004632,5,0.855507,0.859101,0.860929,0.858513,0.002252


In [63]:
grid_results.to_csv(index=False)

'mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_epochs,param_learn_rate,param_neurons,params,split0_test_mse,split1_test_mse,split2_test_mse,mean_test_mse,std_test_mse,rank_test_mse,split0_train_mse,split1_train_mse,split2_train_mse,mean_train_mse,std_train_mse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,split0_train_mae,split1_train_mae,split2_train_mae,mean_train_mae,std_train_mae,split0_test_r2,split1_test_r2,split2_test_r2,mean_test_r2,std_test_r2,rank_test_r2,split0_train_r2,split1_train_r2,split2_train_r2,mean_train_r2,std_train_r2\n57.547768433888756,4.313871735459615,0.602834145228068,0.048697412629645394,500,150,0.0005,100,"{\'batch_size\': 500, \'epochs\': 150, \'learn_rate\': 0.0005, \'neurons\': 100}",0.0016265625599771738,0.00116056390106678,0.0017558688996359706,0.0015143317868933082,0.00025566099969897324,3,0.0014312304556369781,0.001614388427697122,0.00135353475343436,0.0014663845455894868,0.

In [67]:
df_sc_2['O3'].max()

284.44

In [68]:
df_sc_2['O3'].min()

0.0

In [76]:
lista=[0.001466384546,
0.001470751595,
0.001462738418,
0.001467104536,
0.001487625297,
0.001468392788,
]

In [77]:
for i in lista:
  print(math.sqrt(i))

0.03829340081528409
0.03835037933319565
0.03824576339936229
0.03830280062867466
0.038569745876787936
0.038319613620181506


In [73]:
import math